In [118]:
from datetime import timedelta
from collections import defaultdict
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [119]:
df = pd.read_csv("./librivox_splits_times.csv")

In [120]:
df.head()

,author_id,book_id,language,seconds,split
0,https://librivox.org/reader/5623,1c707000335619d7768341cc5b36d74e,Arabic,2212,test
1,https://librivox.org/reader/1618,cffdcff856afd3b8bbc1f3e2d29adf0f,Arabic,936,test
2,https://librivox.org/reader/1474,098c716188512f48a8e96c2f91820a51,Chinese,110,test
3,https://librivox.org/reader/9559,37614b5e1ec0f3d5391a1af0249663fe,Chinese,120,test
4,https://librivox.org/reader/2245,7fd126e9cceffa4e9ee4bd9ab4827d42,Chinese,430,test


In [121]:
# make a table with langugage, split, n_authors, time
rows = defaultdict(lambda : timedelta(seconds=0, hours=0, minutes=0))
authors_rows = defaultdict(set)

for _, row in df.iterrows():
    key = (row["language"], row["split"]) 
    rows[key] += timedelta(seconds=row["seconds"])
    authors_rows[key].add(row["author_id"])
    
data = []
for (language, split) in rows:
    data.append({
        "language": language,
        "split": split,
        "time": rows[(language, split)],
        "n_authors": len(authors_rows[(language, split)])
    })

stats = pd.DataFrame(data)

In [122]:
stats.sort_values(by=["time"], inplace=True)

In [123]:
stats[stats["split"] == "train"]

,language,n_authors,split,time
23,Arabic,1,train,00:46:52
36,Korean,1,train,00:48:30
44,Tagalog,1,train,00:50:10
35,Japanese,1,train,00:51:24
25,Danish,1,train,00:52:00
33,Hebrew,1,train,00:54:00
43,Swedish,2,train,01:02:00
28,Esperanto,1,train,01:10:48
32,Greek,2,train,01:16:40
41,Russian,5,train,01:18:40


In [124]:
stats[stats["split"] == "val"]

,language,n_authors,split,time
50,English,6,val,00:27:20
60,Latin,6,val,00:27:21
54,German,6,val,00:29:00
53,French,6,val,00:29:12
63,Portuguese,6,val,00:30:02
57,Italian,6,val,00:30:30
62,Polish,5,val,00:30:40
52,Finnish,5,val,00:30:50
68,Tamil,2,val,00:31:20
49,Dutch,6,val,00:32:34


In [125]:
stats[stats["split"] == "test"]

,language,n_authors,split,time
19,Spanish,6,test,00:28:00
4,English,6,test,00:28:30
8,German,6,test,00:29:20
14,Latin,6,test,00:30:40
16,Polish,5,test,00:31:30
1,Chinese,5,test,00:31:50
6,Finnish,5,test,00:31:50
22,Tamil,2,test,00:32:50
11,Italian,6,test,00:32:50
3,Dutch,6,test,00:33:09


In [171]:
def checker(input):
    if input == "train":
        return 1
    elif input == "val":
        return 2
    else:
        return 3

stats['temp'] = stats['split'].apply(lambda _: checker(_))
stats.head()

,language,n_authors,split,time,seconds,temp
50,English,6,val,00:27:20,1640.0,2
60,Latin,6,val,00:27:21,1641.0,2
19,Spanish,6,test,00:28:00,1680.0,3
4,English,6,test,00:28:30,1710.0,3
54,German,6,val,00:29:00,1740.0,2


In [182]:
df = stats.set_index(['language', 'temp', 'split']).sort_index(level=['language', 'temp']).reset_index(level='temp').drop(labels="temp", axis=1)

n_authors     time  seconds
language     split                             
Arabic       train          1 00:46:52   2812.0
             val            2 00:50:40   3040.0
             test           2 00:52:28   3148.0
Chinese      train          4 01:24:44   5084.0
             val            5 00:33:30   2010.0
             test           5 00:31:50   1910.0
Danish       train          1 00:52:00   3120.0
             val            2 00:47:50   2870.0
             test           2 00:50:10   3010.0
Dutch        train          5 01:24:20   5060.0
             val            6 00:32:34   1954.0
             test           6 00:33:09   1989.0
English      train          5 01:34:10   5650.0
             val            6 00:27:20   1640.0
             test           6 00:28:30   1710.0
Esperanto    train          1 01:10:48   4248.0
             val            2 00:37:39   2259.0
             test           2 00:41:40   2500.0
Finnish      train          4 01:27:20   5240.0
             val            5 00:30:50   1850.0
             test           5 00:31:50   1910.0
French       train          5 01:26:54   5214.0
             val            6 00:29:12   1752.0
             test           6 00:33:52   2032.0
German       train          5 01:31:40   5500.0
             val            6 00:29:00   1740.0
             test           6 00:29:20   1760.0
Greek        train          2 01:16:40   4600.0
             val            3 00:39:23   2363.0
             test           3 00:34:00   2040.0
...                       ...      ...      ...
Korean       train          1 00:48:30   2910.0
             val            2 00:50:55   3055.0
             test           2 00:50:38   3038.0
Latin        train          5 01:32:00   5520.0
             val            6 00:27:21   1641.0
             test           6 00:30:40   1840.0
Multilingual train          1 01:19:15   4755.0
             val            2 00:33:39   2019.0
             test           2 00:37:09   2229.0
Polish       train          4 01:27:54   5274.0
             val            5 00:30:40   1840.0
             test           5 00:31:30   1890.0
Portuguese   train          5 01:26:40   5200.0
             val            6 00:30:02   1802.0
             test           6 00:33:16   1996.0
Russian      train          5 01:18:40   4720.0
             val            6 00:36:50   2210.0
             test           6 00:34:30   2070.0
Spanish      train          5 01:25:46   5146.0
             val            6 00:36:20   2180.0
             test           6 00:28:00   1680.0
Swedish      train          2 01:02:00   3720.0
             val            3 00:42:30   2550.0
             test           3 00:45:30   2730.0
Tagalog      train          1 00:50:10   3010.0
             val            2 00:47:15   2835.0
             test           2 00:50:29   3029.0
Tamil        train          1 01:25:50   5150.0
             val            2 00:31:20   1880.0
             test           2 00:32:50   1970.0

[69 rows x 3 columns]

In [127]:
stats.to_csv("librivox_stats.csv", index=False)

In [128]:
# stats['seconds'] = stats['time'].apply(lambda _: _.total_seconds())
# stats['seconds'][1]

1910.0

In [135]:
# df = pd.pivot_table(stats, index=['language', 'split'], columns=[], values=['n_authors', 'seconds'])
# df.set_index(['language', 'split'], inplace=True)

In [144]:
# df.time = df.seconds.apply(lambda _: str(timedelta(seconds=_))[:7])

In [149]:
huh = df.to_latex()